In [1]:
import sqlite3

# Connect to SQLite database (or create if not exists)
conn = sqlite3.connect('emaildb.sqlite')
cur = conn.cursor()

# Drop table if exists (to ensure a fresh start)
cur.execute('DROP TABLE IF EXISTS Counts')

# Create table with specified schema
cur.execute('CREATE TABLE Counts (org TEXT, count INTEGER)')

# Prompt user for file name, defaulting to mbox.txt
filename = input('Enter file name: ')
if len(filename) < 1:
    filename = 'mbox.txt'

# Open the file
try:
    fh = open(filename)
except:
    print('File cannot be opened:', filename)
    quit()

# Loop through lines of the file
for line in fh:
    # Look for lines starting with 'From: '
    if not line.startswith('From: '): 
        continue
    # Split the line into words
    pieces = line.split()
    # Extract email address
    email = pieces[1]
    # Extract domain from email address
    domain = email.split('@')[1]
    # Check if domain exists in database, if not, insert, else update count
    cur.execute('SELECT count FROM Counts WHERE org = ?', (domain,))
    row = cur.fetchone()
    if row is None:
        cur.execute('INSERT INTO Counts (org, count) VALUES (?, 1)', (domain,))
    else:
        count = row[0] + 1
        cur.execute('UPDATE Counts SET count = ? WHERE org = ?', (count, domain))

# Commit changes to the database
conn.commit()

# Retrieve data and print counts
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC'

print("Counts:")
for row in cur.execute(sqlstr):
    print(row[0], row[1])

# Close cursor and connection
cur.close()


Enter file name: mbox.txt
Counts:
iupui.edu 8
umich.edu 7
uct.ac.za 6
media.berkeley.edu 4
caret.cam.ac.uk 1
gmail.com 1
